In [1]:
import keras_tuner
from tensorflow import keras
from keras_tuner.tuners import RandomSearch
import pandas as pd

Using TensorFlow backend


2024-01-17 09:18:15.722898: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:18:15.922636: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 09:18:15.922682: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 09:18:15.923928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 09:18:16.045876: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 09:18:16.047748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
#adv_df = pd.read_csv("./data/shadow_model_outputs.csv")
adv_df = pd.read_csv("data/shadow_model_outputs_proba-new.csv")
adv_ddf_shuffled = adv_df.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y = adv_ddf_shuffled["y"]
adv_X = adv_ddf_shuffled.drop(columns=["y"])

In [4]:
adv_df_test = pd.read_csv("data/shadow_model_outputs_proba-new_test_set.csv")
#adv_df_test = pd.read_csv("./data/shadow_model_outputs_test_set.csv")
adv_ddf_shuffled_test = adv_df_test.sample(frac=1, random_state=1).reset_index(drop=True)#.drop(columns=["Unnamed: 0"])
adv_y_test = adv_ddf_shuffled_test["y"]
adv_X_test = adv_ddf_shuffled_test.drop(columns=["y"])

In [13]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(keras.layers.Dense(hp.Int('units_dense0', min_value=26, max_value=46, step=2)))
        #if hp.Boolean("relu_dense0"):
        #    model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.01, max_value=0.03, step=0.01)))
        model.add(keras.layers.Dense(hp.Int('units_dense1', min_value=12, max_value=20, step=2)))
        #if hp.Boolean("relu_dense1"):
        #    model.add(keras.layers.Activation(keras.activations.relu))
        model.add(keras.layers.Dense(1))
        model.compile(optimizer="adam", loss=keras.losses.MeanSquaredError(), metrics=keras.metrics.R2Score())
        return model
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [40, 64]),
            **kwargs,
        )


In [14]:
from keras_tuner import Objective
tuner = RandomSearch(
    MyHyperModel(),
    #objective=keras_tuner.Objective('val_r2_score', direction='max'),
    objective = Objective(name="val_r2_score", direction="max"),
    max_trials = 50,
    executions_per_trial = 4,
    tuner_id="17.01.regression.adversary.cgpt"
)

In [15]:
tuner.search(adv_X, adv_y, epochs=200, validation_data=(adv_X_test, adv_y_test), callbacks=[keras.callbacks.EarlyStopping('val_r2_score', patience=30)])

Trial 50 Complete [00h 01m 07s]
val_r2_score: 0.06523823738098145

Best val_r2_score So Far: 0.26345284283161163
Total elapsed time: 00h 54m 44s


In [16]:
models = tuner.get_best_models(num_models=5)

In [17]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_r2_score", direction="max")

Trial 19 summary
Hyperparameters:
units_dense0: 38
dropout_rate: 0.01
units_dense1: 12
batch_size: 40
Score: 0.26345284283161163

Trial 44 summary
Hyperparameters:
units_dense0: 38
dropout_rate: 0.01
units_dense1: 20
batch_size: 40
Score: 0.23251301050186157

Trial 14 summary
Hyperparameters:
units_dense0: 30
dropout_rate: 0.01
units_dense1: 14
batch_size: 40
Score: 0.2296116054058075

Trial 03 summary
Hyperparameters:
units_dense0: 32
dropout_rate: 0.01
units_dense1: 16
batch_size: 40
Score: 0.184507817029953

Trial 06 summary
Hyperparameters:
units_dense0: 40
dropout_rate: 0.01
units_dense1: 18
batch_size: 64
Score: 0.14947256445884705

Trial 20 summary
Hyperparameters:
units_dense0: 30
dropout_rate: 0.02
units_dense1: 12
batch_size: 40
Score: 0.14485591650009155

Trial 38 summary
Hyperparameters:
units_dense0: 34
dropout_rate: 0.02
units_dense1: 12
batch_size: 40
Sco

In [10]:
best_model = models[0]

In [11]:
best_model.build(input_shape=(None,adv_X.shape[1],))

In [37]:
best_model.layers[1].rate

0.01

In [38]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 46)                920138    
                                                                 
 dropout (Dropout)           (None, 46)                0         
                                                                 
 dense_1 (Dense)             (None, 18)                846       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 921003 (3.51 MB)
Trainable params: 921003 (3.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
best_model.save("adv_new_0.26_test_r2")
#X_train, y_train, X_test, y_test, _, _ = data_train_test(train_size=0.7)

INFO:tensorflow:Assets written to: adv_new_0.26_test_r2/assets


INFO:tensorflow:Assets written to: adv_new_0.26_test_r2/assets


In [40]:
best_model.predict(adv_X_test)

57/57 [==============================] - 0s 2ms/step


array([[0.5596606 ],
       [0.4603508 ],
       [0.4807027 ],
       ...,
       [0.62426335],
       [0.4381116 ],
       [0.584784  ]], dtype=float32)

In [41]:
adv_y_test

0       0.8
1       0.3
2       0.4
3       0.7
4       0.5
       ... 
1795    0.5
1796    0.9
1797    0.6
1798    0.2
1799    0.6
Name: y, Length: 1800, dtype: float64